In [1]:
!pip install stopwordsiso

Preprocessing Pipeline HSA

Imports

In [2]:
import os
import glob
import re
import time
from tqdm import tqdm
from lxml import etree
import pandas as pd
import stopwordsiso as stopwords

c:\Users\michael\AppData\Local\anaconda3\envs\tei-mapper\Lib\site-packages\stopwordsiso\_core.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
TEI_NS = {"tei": "http://www.tei-c.org/ns/1.0"}
XML_NS = "{http://www.w3.org/XML/1998/namespace}"

In [4]:
def extract_paragraph_metadata_from_files(file_list):
    all_rows = []

    def get_id_and_surname(pn_nodes):
        if not pn_nodes:
            return None, None
        pn = pn_nodes[0]
        pid = pn.get("corresp")
        sname = pn.xpath("normalize-space(tei:surname)", namespaces=TEI_NS) or None
        return pid, sname

    def text_of(elem):
        # take all text nodes not inside <tei:note>, then collapse whitespace
        parts = elem.xpath(".//text()[not(ancestor::tei:note)]", namespaces=TEI_NS)
        return " ".join("".join(parts).split())

    def get_subject_pref_label(root):
        label = root.xpath(
            "//tei:profileDesc/tei:textClass/tei:keywords[@scheme='https://gams.uni-graz.at/o:hsa.subjects']"
            "/tei:term[@type='skos:Concept']/tei:term[@type='skos:prefLabel']/text()",
            namespaces=TEI_NS,
        )
        return label[0].strip() if label else None

    for file_path in file_list:
        tree = etree.parse(file_path)
        root = tree.getroot()

        sender_pn = root.xpath(
            "//tei:teiHeader//tei:correspDesc/tei:correspAction[@type='sent']/tei:persName",
            namespaces=TEI_NS,
        )
        recv_pn = root.xpath(
            "//tei:teiHeader//tei:correspDesc/tei:correspAction[@type='received']/tei:persName",
            namespaces=TEI_NS,
        )
        sender_id, sender_surname = get_id_and_surname(sender_pn)
        receiver_id, receiver_surname = get_id_and_surname(recv_pn)

        # Prefer language of ORIGINAL block
        lang = root.xpath("//tei:body/tei:div[@type='letter' and @subtype='original']/@xml:lang", namespaces=TEI_NS)
        if not lang:
            lang = root.xpath("//tei:body/tei:div[@subtype='original']/@xml:lang", namespaces=TEI_NS)
        lang = lang[0] if lang else None

        subject_label = get_subject_pref_label(root)

        date_sent = root.xpath(
            "//tei:teiHeader//tei:correspDesc/tei:correspAction[@type='sent']/tei:date/@when",
            namespaces=TEI_NS
        )
        date_sent = date_sent[0] if date_sent else None

        # Collect ONLY original letters 
        original_divs = root.xpath(
            "//tei:text/tei:body//tei:div[@type='letter' and @subtype='original']",
            namespaces=TEI_NS
        )

        if not original_divs:
            continue

        
        def pick_base_letter_num(root, divs, file_path):
            #  Canonical PID:  "o:hsa.letter.6060"
            pid_vals = root.xpath(
                "//tei:fileDesc/tei:publicationStmt/tei:idno[@type='PID']/text()",
                namespaces=TEI_NS
            )
            if pid_vals:
                pid_text = pid_vals[0].strip()
                m = re.search(r'letter\.([A-Za-z0-9_-]+)$', pid_text)
                if m:
                    return m.group(1)       # -> "L.6060"
                return pid_text

            # Fallback to correspDesc @ref: "...o:hsa.letter.6060"
            ref_vals = root.xpath("//tei:profileDesc/tei:correspDesc/@ref", namespaces=TEI_NS)
            if ref_vals:
                m = re.search(r'letter\.([A-Za-z0-9_-]+)$', ref_vals[0])
                if m:
                    return m.group(1)


            for d in divs:
                xid = d.get(f"{XML_NS}id", "")
                if xid and xid.startswith("L.") and "." in xid:
                    return xid

            # deterministic fallback
            base = os.path.splitext(os.path.basename(file_path))[0]
            return f"fallback_{base}"

        letter_base = pick_base_letter_num(root, original_divs, file_path)

        
        para_idx = 1
        for div in original_divs:
            paragraphs = div.xpath(".//tei:p", namespaces=TEI_NS)
            for p in paragraphs:
                txt = text_of(p).strip()
                if len(txt) < 1:   # skip empty <p/> elements, adjustable to skip shorter paragraphs
                    continue

                pid = f"L.{letter_base}-{para_idx}"
                para_idx += 1

                all_rows.append({
                    "source_file": os.path.basename(file_path),
                    "pid": pid,
                    "sender_id": sender_id,
                    "sender": sender_surname,
                    "receiver_id": receiver_id,
                    "receiver": receiver_surname,
                    "date": date_sent,
                    "text": txt,
                    "language": lang,
                    "keywords": subject_label,
        })

    df = pd.DataFrame(all_rows, columns=[
        "source_file", "pid",
        "sender_id", "sender",
        "receiver_id", "receiver", "date",
        "text", "language", "keywords"
    ])
    return all_rows, df

In [5]:
def count_words(text):
    if not text:
        return 0
    return len(text.strip().split())


In [6]:
def filter_long_paragraphs(df, threshold=256):

    if "word_count" not in df.columns:
        raise ValueError("DataFrame must have a 'word_count' column.")

    long_df = df[df["word_count"] > threshold].copy()
    count = len(long_df)
    return long_df, count

In [7]:
def split_long_paragraphs(long_df: pd.DataFrame, chunk_size: int = 511):

    required_cols = {"pid", "text"}
    missing = required_cols - set(long_df.columns)
    if missing:
        raise ValueError(f"Missing required column(s): {', '.join(sorted(missing))}")

    all_sections = []
    meta_cols = [c for c in long_df.columns if c not in ("pid", "text")]

    for _, row in long_df.iterrows():
        parent_pid = row["pid"]
        text = row["text"] or ""
        words = text.strip().split()
        if not words:
            continue

        section_num = 0
        for start in range(0, len(words), chunk_size):
            section_num += 1
            chunk_words = words[start:start + chunk_size]
            chunk_text = " ".join(chunk_words)

            section_id = f"{parent_pid}-{section_num}"  # e.g., L.1-2-1

            section_row = {
                "section_id": section_id,
                "parent_pid": parent_pid,
                "section_number": section_num,
                "section_text": chunk_text,
                "section_word_count": len(chunk_words),
            }

            for c in meta_cols:
                section_row[c] = row[c]

            all_sections.append(section_row)

    df_sections = pd.DataFrame(all_sections, columns=[
        "section_id", "parent_pid", "section_number",
        "section_text", "section_word_count", *meta_cols
    ])
    return all_sections, df_sections

In [8]:
def replace_long_paragraphs_with_sections(df, chunk_size=511, threshold=511, pad_sections=False):

    # sanity 
    if not df.columns.is_unique:
        dups = df.columns[df.columns.duplicated()].tolist()
        raise ValueError(f"Input df has duplicate column names: {dups}")

    required_cols = {"pid", "text", "word_count"}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Missing required column(s) in df: {', '.join(sorted(missing))}")


    short_df = df[df["word_count"] <= threshold].copy()
    long_df  = df[df["word_count"] >  threshold].copy()


    _, sections_df = split_long_paragraphs(long_df, chunk_size=chunk_size)

    if pad_sections:
        # zum Beispiel L.x-y-01 
        sections_df["pid"] = sections_df["parent_pid"] + "-" + sections_df["section_number"].astype(str).str.zfill(2)
    else:
        sections_df["pid"] = sections_df["parent_pid"] + "-" + sections_df["section_number"].astype(str)

    sections_df["text"] = sections_df["section_text"]
    sections_df["word_count"]     = sections_df["section_word_count"]

    base_cols = list(df.columns)

    # add cols
    for c in base_cols:
        if c not in sections_df.columns:
            sections_df[c] = pd.NA

    helper_cols = [c for c in sections_df.columns if c not in base_cols]
    if helper_cols:
        sections_df = sections_df.drop(columns=helper_cols)


    sections_df = sections_df[base_cols]

    # concat back 
    combined_df = pd.concat([short_df, sections_df], ignore_index=True, sort=False)
    sort_cols = [c for c in ["source_file", "paragraph_id"] if c in combined_df.columns]
    if sort_cols:
        combined_df = combined_df.sort_values(by=sort_cols).reset_index(drop=True)

    return combined_df


In [9]:

def get_para_per_amount_of_words(df, amount = 511):
    mask = df["text"].apply(lambda text: len(str(text).split()) > amount)
    return df.loc[mask, ["pid", "text"]]

In [10]:
def get_unique_languages(df):
    return sorted(lang for lang in df["language"].dropna().unique())

In [11]:
folder_path = "get"  # <- adjust if needed
file_pattern = "*.xml"  
file_list = glob.glob(os.path.join(folder_path, file_pattern))

In [12]:

rows, df = extract_paragraph_metadata_from_files(file_list)
df["word_count"] = df["text"].apply(count_words)

df_final = replace_long_paragraphs_with_sections(df, chunk_size=511, threshold=511)

print(df_final.head())

         source_file     pid                                      sender_id  \
0   hsa.letter.1.xml   L.1-1  https://gams.uni-graz.at/o:hsa.persons#P.1069   
1   hsa.letter.1.xml   L.1-2  https://gams.uni-graz.at/o:hsa.persons#P.1069   
2  hsa.letter.10.xml  L.10-9  https://gams.uni-graz.at/o:hsa.persons#P.2100   
3  hsa.letter.10.xml  L.10-8  https://gams.uni-graz.at/o:hsa.persons#P.2100   
4  hsa.letter.10.xml  L.10-7  https://gams.uni-graz.at/o:hsa.persons#P.2100   

              sender                                   receiver_id  \
0            Baissac  https://gams.uni-graz.at/o:hsa.persons#P.109   
1            Baissac  https://gams.uni-graz.at/o:hsa.persons#P.109   
2  Machado y Álvarez  https://gams.uni-graz.at/o:hsa.persons#P.109   
3  Machado y Álvarez  https://gams.uni-graz.at/o:hsa.persons#P.109   
4  Machado y Álvarez  https://gams.uni-graz.at/o:hsa.persons#P.109   

     receiver        date                                               text  \
0  Schuchardt  1885-01-2

In [13]:
languages = get_unique_languages(df)
print(languages)

df_511 = get_para_per_amount_of_words(df_final, amount = 511)
print(df_511)

['ar', 'ca', 'cy', 'da', 'de', 'en', 'es', 'eu', 'fr', 'ft', 'hu', 'idb', 'io', 'it', 'la', 'lad', 'ms', 'nl', 'pap', 'pt', 'ro', 'roa']
Empty DataFrame
Columns: [pid, text]
Index: []


In [14]:

def sort_by_pid(df):

    def natural_key(pid):
        return [int(s) if s.isdigit() else s.lower() 
                for s in re.split(r'(\d+)', pid)]
    
    return df.sort_values(by="pid", key=lambda col: col.map(natural_key)).reset_index(drop=True)

In [15]:
drop = {"source_file"}
df_final = df_final.drop(columns=drop)
df_final = sort_by_pid(df_final)
df_final.head()

,pid,sender_id,sender,receiver_id,receiver,date,text,language,keywords,word_count
0,L.1-1,https://gams.uni-graz.at/o:hsa.persons#P.1069,Baissac,https://gams.uni-graz.at/o:hsa.persons#P.109,Schuchardt,1885-01-20,Ma Doudou vous envoie une petite brochure jaun...,fr,None,17
1,L.1-2,https://gams.uni-graz.at/o:hsa.persons#P.1069,Baissac,https://gams.uni-graz.at/o:hsa.persons#P.109,Schuchardt,1885-01-20,Nous sommes anxieux l’un et l’autre d’avoir de...,fr,None,31
2,L.3-1,https://gams.uni-graz.at/o:hsa.persons#P.1068,Bähr,https://gams.uni-graz.at/o:hsa.persons#P.109,Schuchardt,1924-04-22,Bereits 5 Tage nach Abgang meines letzten Brie...,de,Revue de Linguistique Romane,124
3,L.3-2-1,https://gams.uni-graz.at/o:hsa.persons#P.1068,Bähr,https://gams.uni-graz.at/o:hsa.persons#P.109,Schuchardt,1924-04-22,"Was die deutsche Verlagswelt angeht, so muss i...",de,Revue de Linguistique Romane,511
4,L.3-2-2,https://gams.uni-graz.at/o:hsa.persons#P.1068,Bähr,https://gams.uni-graz.at/o:hsa.persons#P.109,Schuchardt,1924-04-22,"sicherlich viel verdankt— erklärt, er wolle di...",de,Revue de Linguistique Romane,46


In [16]:
dupes = df[df.duplicated("pid")]["pid"].unique()
print(dupes)
print(len(dupes))

[]
0


In [17]:
empty_text_rows = df[df["text"].isna() | (df["text"].str.strip() == "")]
print(empty_text_rows)

Empty DataFrame
Columns: [source_file, pid, sender_id, sender, receiver_id, receiver, date, text, language, keywords, word_count]
Index: []


In [18]:
df_final.to_csv("letters_py.csv", index=False, encoding="utf-8")

In [19]:
langs = df_final['language'].unique()
for lang in langs:
    print(lang, stopwords.has_lang(lang))

fr True
de True
es True
eu True
da True
pt True
it True
nl True
ft False
en True
ca True
la True
ro True
idb False
cy False
pap False
io False
hu True
ar True
lad False
ms True
roa False
